In [2]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")
import json

In [3]:
with open("fixed_log.log") as f:
    log = f.readlines()

In [4]:
#for i, entry in enumerate(log):
 #   requests.post(f"https://localhost:9200/gluttonhoneypot/_doc/{i+1}", json=json.loads(entry), auth=('admin', 'admin'), verify=False)

In [5]:
log[3:5]

['{"level":"info","ts":1673794463.1364226,"msg":"IP from a known scanner","sensorID":"cd00083f-1221-429d-9d07-49cc5e1c2005","host":"167.94.138.61","scanner":"censys","dest_port":"53482"}\n',
 '{"level":"info","ts":1673794464.202223,"msg":"IP from a known scanner","sensorID":"cd00083f-1221-429d-9d07-49cc5e1c2005","host":"167.94.138.61","scanner":"censys","dest_port":"53482"}\n']

In [6]:
list_of_dicts = []
for line in log[3:]:
    list_of_dicts.append(json.loads(line))

In [7]:
df = pd.DataFrame(list_of_dicts)

In [8]:
df.head()

,level,ts,msg,sensorID,host,scanner,dest_port,handler,src_ip,src_port,...,password,message,direction,error,payload_hash,peer_id,inf_hash,sha256,source,sha
0,info,1.673794e+09,IP from a known scanner,cd00083f-1221-429d-9d07-49cc5e1c2005,167.94.138.61,censys,53482,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,info,1.673794e+09,IP from a known scanner,cd00083f-1221-429d-9d07-49cc5e1c2005,167.94.138.61,censys,53482,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,info,1.673794e+09,IP from a known scanner,cd00083f-1221-429d-9d07-49cc5e1c2005,167.94.138.61,censys,53482,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,info,1.673794e+09,"running Go routines: 35, open files: 8",cd00083f-1221-429d-9d07-49cc5e1c2005,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,info,1.673794e+09,IP from a known scanner,cd00083f-1221-429d-9d07-49cc5e1c2005,167.94.138.61,censys,53482,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
pd.set_option("display.max_rows", None)
df.dest_port.value_counts().head(10)

23       2416231
2323      177774
3399       41689
37215      34065
23231       4162
80          3808
21          3255
8080        2802
8888        1478
6379        1462
Name: dest_port, dtype: int64

In [18]:
# the top 100 attacker IPs make up 72% of the registered hits
(df.src_ip.value_counts().to_frame().head(100).sum() / len(list_of_dicts)) * 100

src_ip    72.164882
dtype: float64

In [30]:
df.src_ip.value_counts().head(10)

194.180.49.134     1444948
78.135.85.216       469213
171.22.30.173       183565
85.209.134.96        73187
170.250.201.168      47101
54.39.186.162        41618
195.133.40.73        14987
107.189.6.194         8608
14.33.109.148         8489
185.225.73.79         7530
Name: src_ip, dtype: int64

In [31]:
# identify attacker IP locations (https://www.hostip.info/use.php)
for ip in df.src_ip.value_counts().head(10).index.to_list():
    r = requests.get(f"https://api.hostip.info/get_html.php?ip={ip}")
    print(r.text)

Country: EUROPEAN UNION (EU)
City: (Unknown city)
IP: 194.180.49.134

Country: GERMANY (DE)
City: Berlin
IP: 78.135.85.216

Country: EUROPEAN UNION (EU)
City: (Unknown city)
IP: 171.22.30.173

Country: (Unknown Country?) (XX)
City: (Unknown City?)
IP: 85.209.134.96

Country: UNITED STATES (US)
City: (Unknown city)
IP: 170.250.201.168

Country: UNITED STATES (US)
City: (Unknown city)
IP: 54.39.186.162

Country: RUSSIAN FEDERATION (RU)
City: (Unknown city)
IP: 195.133.40.73

Country: (Unknown Country?) (XX)
City: (Unknown City?)
IP: 107.189.6.194

Country: UNITED STATES (US)
City: (Unknown city)
IP: 14.33.109.148

Country: (Unknown Country?) (XX)
City: (Unknown City?)
IP: 185.225.73.79



# #1 attacker IP reverse lookup
![image](top_attacker_ip.png)

`poweredbycrazy.ru` huh?
I assume this is a Digitalocean droplet since the IP location is in the Netherlands

lets check how long the page has been registered

virustotal: https://www.virustotal.com/gui/domain/poweredbycrazy.ru/relations

crt.sh: https://crt.sh/?q=poweredbycrazy.ru